# YOLOv8 Pretrained Detection

**Week 14 - Module 5: Object Detection Models**

**Estimated Time:** 20 minutes

## Learning Objectives
- Use YOLOv8 for real-time object detection
- Detect objects in images and videos
- Understand confidence and NMS thresholds
- Compare different YOLO model sizes

---

## 1. Setup and Installation

We'll use the **Ultralytics** library, which provides a clean, modern interface for YOLOv8.

### Installation
```bash
pip install ultralytics
```

### What's Included?
- Pretrained models (YOLOv8n, YOLOv8s, YOLOv8m, YOLOv8l, YOLOv8x)
- Easy-to-use API
- Built-in visualization
- Support for images, videos, and webcam

In [ ]:
# Install ultralytics (if not already installed)
!pip install -q ultralytics opencv-python matplotlib pillow

# Import libraries
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
import time

print("✅ Setup complete!")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Load YOLOv8 Model

### Available Model Sizes

| Model | Parameters | Size | Speed | mAP |
|-------|-----------|------|-------|-----|
| YOLOv8n | 3.2M | 6.3 MB | Fastest | 37.3% |
| YOLOv8s | 11.2M | 21.5 MB | Fast | 44.9% |
| YOLOv8m | 25.9M | 49.7 MB | Medium | 50.2% |
| YOLOv8l | 43.7M | 83.7 MB | Slow | 52.9% |
| YOLOv8x | 68.2M | 130.5 MB | Slowest | 53.9% |

**Recommendation**: Start with YOLOv8n (nano) for learning and prototyping.

In [ ]:
# Load YOLOv8 nano model (smallest, fastest)
model = YOLO('yolov8n.pt')

print("✅ Model loaded successfully!")
print(f"\n📊 Model Info:")
print(f"Model type: YOLOv8n (Nano)")
print(f"Number of classes: {len(model.names)}")
print(f"\n🏷️ First 10 classes:")
for i, name in list(model.names.items())[:10]:
    print(f"  {i}: {name}")

print("\n...and 70 more classes (trained on COCO dataset)")

## 3. Single Image Detection

Let's detect objects in a sample image.

In [ ]:
# Download sample image
image_url = 'https://ultralytics.com/images/bus.jpg'
urllib.request.urlretrieve(image_url, 'bus.jpg')

# Load and display original image
image = cv2.imread('bus.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.title('Original Image', fontsize=16, fontweight='bold')
plt.axis('off')
plt.show()

print(f"Image shape: {image_rgb.shape}")

In [ ]:
# Run detection
print("Running YOLO detection...")
start_time = time.time()
results = model(image_rgb, verbose=False)
inference_time = (time.time() - start_time) * 1000  # Convert to ms

print(f"✅ Detection complete in {inference_time:.2f} ms")

# Display results
result_img = results[0].plot()

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

axes[0].imshow(image_rgb)
axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(result_img)
axes[1].set_title(f'YOLO Detections ({inference_time:.1f} ms)', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# Print detection details
print("\n🎯 Detection Details:")
print("="*80)
print(f"{'Class':<15} {'Confidence':<12} {'Bounding Box (x1, y1, x2, y2)':<40}")
print("="*80)

for box in results[0].boxes:
    cls_id = int(box.cls[0])
    conf = float(box.conf[0])
    bbox = box.xyxy[0].cpu().numpy()
    class_name = model.names[cls_id]
    
    print(f"{class_name:<15} {conf:<12.3f} [{bbox[0]:>6.1f}, {bbox[1]:>6.1f}, {bbox[2]:>6.1f}, {bbox[3]:>6.1f}]")

print("="*80)
print(f"Total detections: {len(results[0].boxes)}")
print(f"Inference time: {inference_time:.2f} ms")
print(f"FPS: {1000/inference_time:.1f}")

## 4. Batch Image Detection

Process multiple images at once.

In [ ]:
# Download multiple sample images
image_urls = [
    ('https://ultralytics.com/images/bus.jpg', 'bus.jpg'),
    ('https://ultralytics.com/images/zidane.jpg', 'zidane.jpg'),
]

# Download images
for url, filename in image_urls:
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"✓ Downloaded {filename}")
    except:
        print(f"✗ Failed to download {filename}")

# Load images
image_paths = ['bus.jpg', 'zidane.jpg']
images = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]

# Run batch detection
print("\nRunning batch detection...")
results = model(images, verbose=False)

# Display results in grid
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, (img, result) in enumerate(zip(images, results)):
    # Original image
    axes[idx*2].imshow(img)
    axes[idx*2].set_title(f'Original Image {idx+1}', fontsize=12, fontweight='bold')
    axes[idx*2].axis('off')
    
    # Detection result
    result_img = result.plot()
    axes[idx*2 + 1].imshow(result_img)
    axes[idx*2 + 1].set_title(f'Detections ({len(result.boxes)} objects)', fontsize=12, fontweight='bold')
    axes[idx*2 + 1].axis('off')

plt.tight_layout()
plt.show()

# Summary statistics
print("\n📊 Batch Detection Summary:")
print("="*60)
for idx, result in enumerate(results):
    unique_classes = set([model.names[int(box.cls[0])] for box in result.boxes])
    print(f"Image {idx+1}: {len(result.boxes)} detections - {', '.join(unique_classes)}")

## 5. Confidence Threshold Tuning

The **confidence threshold** determines which detections to keep.

- **High threshold (0.7-0.9)**: Only very confident detections (fewer false positives)
- **Low threshold (0.1-0.3)**: More detections (more false positives)
- **Default**: Usually 0.25-0.5

Let's see the effect of different thresholds.

In [ ]:
# Test different confidence thresholds
thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]

# Load image
image = cv2.cvtColor(cv2.imread('bus.jpg'), cv2.COLOR_BGR2RGB)

# Create subplot grid
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Original image
axes[0].imshow(image)
axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Run detection with different thresholds
for idx, threshold in enumerate(thresholds):
    results = model(image, conf=threshold, verbose=False)
    result_img = results[0].plot()
    
    axes[idx + 1].imshow(result_img)
    axes[idx + 1].set_title(
        f'Confidence ≥ {threshold} ({len(results[0].boxes)} detections)',
        fontsize=12, fontweight='bold'
    )
    axes[idx + 1].axis('off')

plt.suptitle('Effect of Confidence Threshold on Detections', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Print statistics
print("\n📊 Confidence Threshold Analysis:")
print("="*60)
print(f"{'Threshold':<12} {'Detections':<12} {'Change':<12}")
print("="*60)

prev_count = 0
for threshold in thresholds:
    results = model(image, conf=threshold, verbose=False)
    count = len(results[0].boxes)
    change = f"{count - prev_count:+d}" if prev_count > 0 else "-"
    print(f"{threshold:<12.1f} {count:<12} {change:<12}")
    prev_count = count

print("\n💡 Key Insight: Higher threshold = Fewer but more confident detections")

## 6. NMS Threshold Tuning

The **NMS (Non-Maximum Suppression) threshold** controls duplicate removal.

- **High IoU threshold (0.7-0.9)**: Keep more overlapping boxes
- **Low IoU threshold (0.3-0.5)**: Aggressive duplicate removal
- **Default**: Usually 0.45-0.5

In [ ]:
# Test different NMS thresholds
nms_thresholds = [0.2, 0.45, 0.7]

# Create subplot grid
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Run detection with different NMS thresholds
for idx, nms_thresh in enumerate(nms_thresholds):
    results = model(image, iou=nms_thresh, conf=0.25, verbose=False)
    result_img = results[0].plot()
    
    axes[idx].imshow(result_img)
    axes[idx].set_title(
        f'NMS IoU Threshold = {nms_thresh}\n({len(results[0].boxes)} detections)',
        fontsize=12, fontweight='bold'
    )
    axes[idx].axis('off')

plt.suptitle('Effect of NMS Threshold on Duplicate Removal', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 NMS Threshold Guide:")
print("  • Low (0.2-0.3): Aggressive removal, fewer overlapping boxes")
print("  • Medium (0.45-0.5): Balanced (default)")
print("  • High (0.7-0.9): Keep more overlapping detections")

## 7. Model Size Comparison

Compare YOLOv8n (nano), YOLOv8s (small), and YOLOv8m (medium) on the same image.

**Trade-off**: Accuracy vs Speed

In [ ]:
# Load different model sizes
model_sizes = ['yolov8n.pt', 'yolov8s.pt']  # Start with 2 for speed
models = {}

print("Loading models...")
for model_name in model_sizes:
    print(f"  Loading {model_name}...")
    models[model_name] = YOLO(model_name)
    print(f"  ✓ {model_name} loaded")

# Compare on same image
image = cv2.cvtColor(cv2.imread('bus.jpg'), cv2.COLOR_BGR2RGB)

# Run inference and measure time
results_comparison = {}
times = {}

for model_name, model in models.items():
    start = time.time()
    result = model(image, verbose=False)
    times[model_name] = (time.time() - start) * 1000  # ms
    results_comparison[model_name] = result[0]

# Visualize comparison
fig, axes = plt.subplots(1, len(model_sizes), figsize=(9 * len(model_sizes), 8))
if len(model_sizes) == 1:
    axes = [axes]

for idx, model_name in enumerate(model_sizes):
    result_img = results_comparison[model_name].plot()
    axes[idx].imshow(result_img)
    
    model_label = model_name.replace('.pt', '').replace('yolov8', 'YOLOv8-').upper()
    axes[idx].set_title(
        f'{model_label}\n{times[model_name]:.1f} ms | {len(results_comparison[model_name].boxes)} detections',
        fontsize=14, fontweight='bold'
    )
    axes[idx].axis('off')

plt.suptitle('Model Size Comparison: Accuracy vs Speed', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

# Print detailed comparison
print("\n📊 Model Comparison Summary:")
print("="*70)
print(f"{'Model':<15} {'Time (ms)':<12} {'FPS':<10} {'Detections':<12}")
print("="*70)

for model_name in model_sizes:
    fps = 1000 / times[model_name]
    detections = len(results_comparison[model_name].boxes)
    model_label = model_name.replace('.pt', '')
    print(f"{model_label:<15} {times[model_name]:<12.2f} {fps:<10.1f} {detections:<12}")

print("="*70)
print("\n💡 Generally: Larger models are slower but more accurate")

## 8. Class-Specific Detection

Sometimes you only want to detect specific classes (e.g., only people, only vehicles).

This is useful for:
- People counting
- Vehicle detection in traffic
- Specific object tracking

In [ ]:
# Detect only specific classes
# COCO class IDs: 0=person, 2=car, 5=bus, 7=truck

# Run full detection first
results_full = model(image, verbose=False)

# Filter to only people (class 0)
results_people = model(image, classes=[0], verbose=False)  # Only person

# Filter to only vehicles (car, bus, truck)
results_vehicles = model(image, classes=[2, 5, 7], verbose=False)  # car, bus, truck

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# All detections
axes[0].imshow(results_full[0].plot())
axes[0].set_title(f'All Classes\n({len(results_full[0].boxes)} detections)', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Only people
axes[1].imshow(results_people[0].plot())
axes[1].set_title(f'Only People\n({len(results_people[0].boxes)} detections)', fontsize=12, fontweight='bold')
axes[1].axis('off')

# Only vehicles
axes[2].imshow(results_vehicles[0].plot())
axes[2].set_title(f'Only Vehicles\n({len(results_vehicles[0].boxes)} detections)', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.suptitle('Class-Specific Detection', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n🎯 Class Filtering Examples:")
print("  model(image, classes=[0])           # Only people")
print("  model(image, classes=[2, 5, 7])     # Only vehicles")
print("  model(image, classes=[16])          # Only dogs")
print("  model(image, classes=[0, 16, 17])   # People, dogs, cats")

## 9. Video Detection (Frame-by-Frame)

YOLO can process videos frame by frame. Here's how to do it efficiently.

In [ ]:
# Note: This is a demonstration. For actual video processing,
# you would use model.predict(source='video.mp4', save=True)

# Simulate video processing on multiple frames (using same image for demo)
print("Simulating video processing...\n")

num_frames = 10
total_time = 0
total_detections = 0

for frame_idx in range(num_frames):
    start = time.time()
    results = model(image, verbose=False)
    frame_time = (time.time() - start) * 1000
    
    total_time += frame_time
    total_detections += len(results[0].boxes)
    
    if frame_idx % 3 == 0:
        print(f"Frame {frame_idx+1:2d}: {frame_time:6.2f} ms | {len(results[0].boxes)} objects")

avg_time = total_time / num_frames
avg_fps = 1000 / avg_time

print("\n" + "="*60)
print(f"Average inference time: {avg_time:.2f} ms/frame")
print(f"Average FPS: {avg_fps:.1f}")
print(f"Total detections: {total_detections}")
print(f"Average detections per frame: {total_detections/num_frames:.1f}")
print("="*60)

print("\n💡 For real video processing, use:")
print("   results = model.predict(source='video.mp4', save=True)")
print("   This will process and save the output video automatically.")

## 10. Webcam Detection (Code Template)

For real-time webcam detection, use this code locally (not in Jupyter).

In [ ]:
# Webcam detection template (run locally, not in Jupyter)

webcam_code = '''
from ultralytics import YOLO
import cv2

# Load model
model = YOLO('yolov8n.pt')

# Open webcam
cap = cv2.VideoCapture(0)

print("Press 'q' to quit")

while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run detection
    results = model(frame, verbose=False)
    
    # Display results
    annotated_frame = results[0].plot()
    cv2.imshow('YOLOv8 Webcam', annotated_frame)
    
    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
'''

print("📹 Webcam Detection Code:")
print("="*60)
print(webcam_code)
print("="*60)
print("\n💡 Save this as 'webcam_detection.py' and run locally!")

## 11. Custom Visualization

Create custom visualizations with specific colors and labels.

In [ ]:
# Custom visualization function
def custom_plot(image, results, model_names):
    """
    Create custom visualization with class-specific colors
    """
    img = image.copy()
    
    # Define colors for different classes (BGR format for cv2)
    class_colors = {
        'person': (0, 255, 0),      # Green
        'car': (255, 0, 0),         # Blue
        'bus': (0, 0, 255),         # Red
        'truck': (255, 255, 0),     # Cyan
    }
    
    for box in results.boxes:
        # Get box coordinates
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        
        # Get class info
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        class_name = model_names[cls_id]
        
        # Get color (default to yellow if class not in dict)
        color = class_colors.get(class_name, (0, 255, 255))
        
        # Draw box
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
        
        # Prepare label
        label = f"{class_name} {conf:.2f}"
        
        # Draw label background
        (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
        cv2.rectangle(img, (x1, y1 - label_h - 10), (x1 + label_w, y1), color, -1)
        
        # Draw label text
        cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    return img

# Apply custom visualization
results = model(image, verbose=False)
custom_img = custom_plot(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), results[0], model.names)
custom_img = cv2.cvtColor(custom_img, cv2.COLOR_BGR2RGB)

# Compare default vs custom
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].imshow(results[0].plot())
axes[0].set_title('Default YOLO Visualization', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(custom_img)
axes[1].set_title('Custom Visualization (Class-Specific Colors)', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n🎨 Custom Colors:")
print("  • Person: Green")
print("  • Car: Blue")
print("  • Bus: Red")
print("  • Truck: Cyan")

## 12. Detection Statistics

Analyze detection results with statistics and visualizations.

In [ ]:
# Run detection
results = model(image, verbose=False)[0]

# Count objects by class
class_counts = {}
confidence_scores = {}

for box in results.boxes:
    cls_id = int(box.cls[0])
    conf = float(box.conf[0])
    class_name = model.names[cls_id]
    
    if class_name not in class_counts:
        class_counts[class_name] = 0
        confidence_scores[class_name] = []
    
    class_counts[class_name] += 1
    confidence_scores[class_name].append(conf)

# Create visualizations
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Bar chart of object counts
classes = list(class_counts.keys())
counts = list(class_counts.values())
colors_bar = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

bars = axes[0].bar(classes, counts, color=colors_bar[:len(classes)])
axes[0].set_xlabel('Object Class', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Object Detection Counts', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Add count labels on bars
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontweight='bold')

# Box plot of confidence scores
conf_data = [confidence_scores[cls] for cls in classes]
bp = axes[1].boxplot(conf_data, labels=classes, patch_artist=True)

# Color the boxes
for patch, color in zip(bp['boxes'], colors_bar[:len(classes)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

axes[1].set_xlabel('Object Class', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Confidence Score', fontsize=12, fontweight='bold')
axes[1].set_title('Confidence Score Distribution', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

# Print detailed statistics
print("\n📊 Detection Statistics:")
print("="*70)
print(f"{'Class':<15} {'Count':<8} {'Avg Conf':<12} {'Min Conf':<12} {'Max Conf':<12}")
print("="*70)

for class_name in classes:
    count = class_counts[class_name]
    confs = confidence_scores[class_name]
    avg_conf = np.mean(confs)
    min_conf = np.min(confs)
    max_conf = np.max(confs)
    
    print(f"{class_name:<15} {count:<8} {avg_conf:<12.3f} {min_conf:<12.3f} {max_conf:<12.3f}")

print("="*70)
print(f"Total detections: {len(results.boxes)}")

## 13. Exercise: Apply to Your Own Images

Now it's your turn! Try these exercises:

### Exercise 1: Basic Detection
1. Upload or download your own image
2. Run YOLOv8 detection on it
3. Print the number and types of objects detected

### Exercise 2: Threshold Tuning
1. Take an image with multiple objects
2. Test confidence thresholds from 0.1 to 0.9
3. Find the optimal threshold for your use case

### Exercise 3: Class-Specific Detection
1. Choose a specific class you're interested in (e.g., "person", "car")
2. Filter detections to show only that class
3. Count how many instances were found

### Exercise 4: Model Comparison
1. Download YOLOv8n, YOLOv8s, and YOLOv8m
2. Run all three on the same image
3. Compare accuracy, speed, and detection quality

### Starter Code

In [ ]:
# Exercise starter code

# TODO: Load your image
# my_image = cv2.imread('path/to/your/image.jpg')
# my_image = cv2.cvtColor(my_image, cv2.COLOR_BGR2RGB)

# TODO: Run detection
# results = model(my_image)

# TODO: Display results
# plt.figure(figsize=(12, 8))
# plt.imshow(results[0].plot())
# plt.axis('off')
# plt.show()

# TODO: Print statistics
# print(f"Total detections: {len(results[0].boxes)}")

print("✏️ Complete the TODOs above to apply YOLO to your own images!")

## 14. Summary

### What We Learned

✅ **Setup**: Install and use Ultralytics YOLOv8

✅ **Basic Detection**: Detect objects in images

✅ **Batch Processing**: Process multiple images efficiently

✅ **Threshold Tuning**: Adjust confidence and NMS thresholds

✅ **Model Comparison**: Trade-off between speed and accuracy

✅ **Class Filtering**: Detect only specific object types

✅ **Video Processing**: Apply YOLO to video streams

✅ **Custom Visualization**: Create custom detection displays

✅ **Statistics**: Analyze detection results

### Key Takeaways

1. **YOLOv8 is powerful yet easy to use** - Just a few lines of code!
2. **Model size matters** - Choose based on your speed/accuracy needs
3. **Thresholds are important** - Tune them for your application
4. **Real-time is achievable** - Even on modest hardware

### Best Practices

- **Start with YOLOv8n** for prototyping
- **Tune thresholds** on your specific data
- **Use class filtering** when you know what you're looking for
- **Monitor FPS** for real-time applications
- **Consider model size** based on deployment constraints

### Preview: Notebook 03 - Custom Dataset Preparation

In the next notebook, we'll learn:
- YOLO dataset format
- How to prepare custom datasets
- Annotation tools and workflows
- Data splitting strategies
- Ready for fine-tuning YOLO on your data!

---

**Next: Prepare your own dataset for training!** 🎯